In [1]:
!pip install PyMuPDF tqdm sentence-transformers faiss-cpu pandas transformers nltk
!pip install -U accelerate bitsandbytes transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [2]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import re
import fitz
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from nltk.tokenize import sent_tokenize

pdf_folder = "/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/Rag model data/irs_eng_pdfs"

In [ ]:
def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    return text.replace("\n", " ").strip()

def split_list(input_list: list, slice_size: int, overlap: int) -> list:
    """Splits a list into overlapping chunks of a given size."""
    return [
        input_list[i:i + slice_size]
        for i in range(0, len(input_list) - overlap, slice_size - overlap)
    ]

num_sentence_chunk_size = 10
overlap = 5
pages_and_chunks = []

for filename in tqdm(os.listdir(pdf_folder)):
    if filename.endswith(".pdf"):
        file_path = os.path.join(pdf_folder, filename)
        file_type = (
            "form" if filename.lower().startswith("f") else
            "instruction" if filename.lower().startswith("i") else
            "publication" if filename.lower().startswith("p") else
            "unknown"
        )
        try:
            doc = fitz.open(file_path)
            for page_number, page in enumerate(doc):
                raw_text = page.get_text()
                formatted_text = text_formatter(raw_text)
                sentences = sent_tokenize(formatted_text)
                sentence_chunks = split_list(sentences, num_sentence_chunk_size, overlap)
                for chunk in sentence_chunks:
                    joined_chunk = " ".join(chunk).replace("  ", " ").strip()
                    joined_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_chunk)
                    pages_and_chunks.append({
                        "file": filename,
                        "file_type": file_type,
                        "page_number": page_number + 1,
                        "sentence_chunk": joined_chunk,
                        "chunk_char_count": len(joined_chunk),
                        "chunk_word_count": len(joined_chunk.split(" ")),
                        "chunk_token_count": len(joined_chunk) / 4
                    })
        except Exception as e:
            print(f"Error reading {filename}: {e}")

df = pd.DataFrame(pages_and_chunks)

df.head()


  0%|          | 0/2214 [00:00<?, ?it/s]

MuPDF error: library error: FT_New_Memory_Face(USYDFC+SourceHanSansSC-Bold): invalid argument



,file,file_type,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,f14824.pdf,form,1,Form 14824 (Rev. 10-2022) Catalog Number 69954...,2258,401,564.50
1,f14824.pdf,form,1,• All taxpayers claiming Head of Household fil...,2340,416,585.00
2,f14824.pdf,form,1,"• In either case, to show where you lived, you...",1956,332,489.00
3,i109495c.pdf,instruction,1,2024 Instructions for Forms 1094-C and 1095-C ...,1094,149,273.50
4,i109495c.pdf,instruction,1,"855, at IRS.gov/irb/2014-13_IRB/ar09.html. For...",627,82,156.75


In [ ]:
csv_save_path = "/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/RAGTaxAssistant/sentence_chunks_df.csv"
df.to_csv(csv_save_path, index=False)

In [5]:
# Loading saved csv
csv_save_path = "/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/RAGTaxAssistant/sentence_chunks_df.csv"
df = pd.read_csv(csv_save_path)
df.head()

,file,file_type,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,f14824.pdf,form,1,Form 14824 (Rev. 10-2022) Catalog Number 69954...,2258,401,564.50,[-9.38651860e-02 4.72247507e-03 6.19203560e-...
1,f14824.pdf,form,1,• All taxpayers claiming Head of Household fil...,2340,416,585.00,[-1.06895670e-01 5.12215160e-02 8.35400149e-...
2,f14824.pdf,form,1,"• In either case, to show where you lived, you...",1956,332,489.00,[-9.33274254e-02 1.76005848e-02 9.84594505e-...
3,i109495c.pdf,instruction,1,2024 Instructions for Forms 1094-C and 1095-C ...,1094,149,273.50,[-7.46698380e-02 1.06692538e-01 -1.70740671e-...
4,i109495c.pdf,instruction,1,"855, at IRS.gov/irb/2014-13_IRB/ar09.html. For...",627,82,156.75,[-2.85072848e-02 7.13037923e-02 -3.76259089e-...


In [6]:
from sentence_transformers import SentenceTransformer
import torch

In [ ]:
# Check for GPU and set appropriate precision
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

# Load a Sentence Transformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2",
                            device=device,
                            trust_remote_code=True)

# Increase batch size for efficient GPU utilization
BATCH_SIZE = 128 if device == "cuda" else 32

# Generate embeddings in batches
text_chunks = df["sentence_chunk"].tolist()
embeddings = model.encode(text_chunks,
                          batch_size=BATCH_SIZE,
                          convert_to_tensor=True,
                          device=device)

# Convert embeddings to NumPy array (if needed)
df["embedding"] = [embedding.cpu().numpy() for embedding in embeddings]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print("device: ", device)
print("dtype: ", dtype)
print("batch size: ", BATCH_SIZE)

device:  cuda
dtype:  torch.float16
batch size:  128


In [ ]:
import faiss

# Convert embeddings to NumPy array
embedding_matrix = np.array(df["embedding"].tolist()).astype('float32')

# Create FAISS index (for L2/Euclidean search)
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

# Save FAISS index
faiss_index_save_path = "/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/RAGTaxAssistant/faiss_index.bin"
faiss.write_index(index, faiss_index_save_path)

df.to_csv(csv_save_path, index=False)

print("Processing Completed! FAISS Index & CSV Saved.")

Processing Completed! FAISS Index & CSV Saved.


In [7]:
import faiss

# Load the FAISS index
faiss_index_save_path = "/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/RAGTaxAssistant/faiss_index.bin"
index = faiss.read_index(faiss_index_save_path)

In [ ]:
# Save the vector embedding model so that we can reuse it later
vector_embedding_model_path = "/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/RAGTaxAssistant/vector_embedding_model_1"
model.save(vector_embedding_model_path)

In [8]:
# Load the vector embedding model from storage

device = "cuda"
dtype = torch.float16
BATCH_SIZE = 128

from sentence_transformers import SentenceTransformer
vector_embedding_model_path = "/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/RAGTaxAssistant/vector_embedding_model_1"
model = SentenceTransformer(vector_embedding_model_path, device=device)

In [9]:
# Function for Similarity Search
def search_similar_text(query_text, top_k=3):
    query_embedding = model.encode([query_text], convert_to_tensor=True).cpu().numpy()
    D, I = index.search(query_embedding, k=top_k)
    similar_chunks = df.iloc[I[0]]["sentence_chunk"].tolist()
    return similar_chunks

# Example Usage
query = "How do I file my tax returns?"
print("Similar Chunks Found:\n", search_similar_text(query))

Similar Chunks Found:
 ['Preparing and filing your tax return. After receiving all your wage and earnings state- ments (Forms W-2, W-2G, 1099-R, 1099-MISC, 1099-NEC, etc. ); unemployment compensation statements (by mail or in a digital format) or other government payment statements (Form 1099-G); and interest, dividend, and retirement statements from banks and investment firms (Forms 1099), you have several options to choose from to prepare and file your tax return. You can prepare the tax return yourself, see if you qualify for free tax preparation, or hire a tax professional to prepare your return. Free options for tax preparation. Go to IRS.gov to see your options for preparing and filing your return online or in your local commun- ity, if you qualify, which include the following. • Direct File. Direct File is a permanent op- tion to file individual federal tax returns on- line—for free—directly and securely with the iRS. Direct File is an option for taxpay- ers in participating sta

In [ ]:
# Get GPU available memory
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 40 GB


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
use_quantization_config = False
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
print(f"[INFO] Using attention implementation: {attn_implementation}")

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

llm_model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype=torch.float16,
                                             quantization_config=quantization_config if use_quantization_config else None,
                                             low_cpu_mem_usage=False,
                                             attn_implementation=attn_implementation)
if not use_quantization_config:
    llm_model.to("cuda")

[INFO] Using attention implementation: sdpa


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
# Save llm model for later reuse

import os
llm_model_save_path = "/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/RAGTaxAssistant/llm_model"
os.makedirs(llm_model_save_path, exist_ok=True)

# Save llm model
llm_model.save_pretrained(llm_model_save_path)

# Save tokenizer
tokenizer.save_pretrained(llm_model_save_path)


('/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/RAGTaxAssistant/llm_model/tokenizer_config.json',
 '/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/RAGTaxAssistant/llm_model/special_tokens_map.json',
 '/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/RAGTaxAssistant/llm_model/tokenizer.json')

In [10]:
# Load llm model and tokenizer from storage

from transformers import AutoTokenizer, AutoModelForCausalLM

llm_model_save_path = "/content/drive/MyDrive/UTD Coursework/Sem 4/AI Agents project/Tax project/RAGTaxAssistant/llm_model"

# Load tokenizer and model from the saved directory
tokenizer = AutoTokenizer.from_pretrained(llm_model_save_path)
llm_model = AutoModelForCausalLM.from_pretrained(llm_model_save_path)
llm_model.to("cuda")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [11]:
def augment_prompt(query_text, context_list):
  context = "- " + "\n- ".join([item for item in context_list])

  base_prompt = """Based on the following context items, please answer the query.
  Give yourself room to think by extracting relevant passages from the context before answering the query.
  Don't return the thinking, only return the answer.
  Make sure your answers are as explanatory as possible.
  \nNow use the following context items to answer the user query:
  {context}
  \nRelevant passages: <extract relevant passages from the context here>
  User query: {query}
  Answer:"""


  base_prompt = base_prompt.format(context=context, query=query_text)
  dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]
  prompt = tokenizer.apply_chat_template(dialogue_template, tokenize=False, add_generation_prompt=True)
  return prompt

In [16]:
input_text = "Who should file form 943?"

similar_chunks = search_similar_text(input_text, 15)
augmented_prompt = augment_prompt(input_text, similar_chunks)

In [17]:
print(augmented_prompt)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Based on the following context items, please answer the query.
  Give yourself room to think by extracting relevant passages from the context before answering the query.
  Don't return the thinking, only return the answer.
  Make sure your answers are as explanatory as possible.
  
Now use the following context items to answer the user query:
  - . . . . . . . . . 4 Who Must File Form 943?
- . . . . 4 Who Must File Form 943? . . . . .
- . . . . 3 Who Must File Form 8962 . . . . . .
- . . . . . . . . . 3 Who Must File Form 8962 .
- 7 Where Should You File Form 941-X? . . . . . . . . .
- . . 6 Where Should You File Form 943-X? . . . . . . .
- . . . . . . . . 5 When Should You File Form 943-X? .
- . . . 5 When Should You File Form 943-X? . . . . . .
- . . . 2 Who Needs To File Form W-2 and Form W-3? . . . . . .
- . . 5 Where Should You File Form 944-X? . . . . . . .
- . . . . . 7 Where Should You File Form 941-X? . . . .
- . . . .

In [18]:
inputs = tokenizer(augmented_prompt, return_tensors="pt").to('cuda')
output_prompt = llm_model.generate(**inputs, temperature=0.7, do_sample=True, max_new_tokens=512)
outputs_decoded = tokenizer.decode(output_prompt[0])
print(f"Model output (decoded):\n{outputs_decoded}\n")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Model output (decoded):
<|begin_of_text|><|begin_of_text|><|start_header_id|>user<|end_header_id|>

Based on the following context items, please answer the query.
  Give yourself room to think by extracting relevant passages from the context before answering the query.
  Don't return the thinking, only return the answer.
  Make sure your answers are as explanatory as possible.
  
Now use the following context items to answer the user query:
  -......... 4 Who Must File Form 943?
-.... 4 Who Must File Form 943?.....
-.... 3 Who Must File Form 8962......
-......... 3 Who Must File Form 8962.
- 7 Where Should You File Form 941-X?.........
-.. 6 Where Should You File Form 943-X?.......
-........ 5 When Should You File Form 943-X?.
-... 5 When Should You File Form 943-X?......
-... 2 Who Needs To File Form W-2 and Form W-3?......
-.. 5 Where Should You File Form 944-X?.......
-..... 7 Where Should You File Form 941-X?....
-........ 2 Who Needs To File Form W-2 and Form W-3?.
-... 4 When Sho

In [ ]:
import re

def clean_llama_output(text):
    # Remove system tokens
    text = re.sub(r"<\|.*?\|>", "", text)  # Removes tokens like <|eot_id|>
    text = re.sub(r"(assistant|user):?", "", text, flags=re.IGNORECASE)
    text = text.strip()
    return text

# Wrapping the final output generation into a function so that we can get multiple outputs when necessary
def generate_rag_answer(question, num_chunks=20, max_tokens=512, temperature=0.7):
    similar_chunks = search_similar_text(question, num_chunks)
    augmented_prompt = augment_prompt(question, similar_chunks)

    inputs = tokenizer(augmented_prompt, return_tensors="pt").to("cuda")
    output_prompt = llm_model.generate(
        **inputs,
        temperature=temperature,
        do_sample=True,
        max_new_tokens=max_tokens
    )
    decoded_output = tokenizer.decode(output_prompt[0])

    # Extract just the answer part
    try:
        raw_answer = decoded_output.split("Answer:")[1].strip()
    except IndexError:
        raw_answer = decoded_output.strip()

    # Clean the raw output
    cleaned_answer = clean_llama_output(raw_answer)

    return {
        "question": question,
        "contexts": similar_chunks,
        "answer": cleaned_answer
    }



In [ ]:

questions = [
    "Who should file form 943?",
    "What is the deadline for tax filing in 2025?"
]


# Calling function for multiple questions
for q in questions:
    rag_result = generate_rag_answer(q)
    print("Rag result is : ", rag_result)
    print("Question is : ", rag_result["question"])
    print("Context is : ", rag_result["contexts"])
    print("Generated answer is : ", rag_result["answer"])
    print("\n\n\n\n")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Rag result is :  {'question': 'Who should file form 943?', 'contexts': ['. . . . . . . . . 4 Who Must File Form 943?', '. . . . 4 Who Must File Form 943? . . . . .', '. . . . 3 Who Must File Form 8962 . . . . . .', '. . . . . . . . . 3 Who Must File Form 8962 .', '7 Where Should You File Form 941-X? . . . . . . . . .', '. . 6 Where Should You File Form 943-X? . . . . . . .', '. . . . . . . . 5 When Should You File Form 943-X? .', '. . . 5 When Should You File Form 943-X? . . . . . .', '. . . 2 Who Needs To File Form W-2 and Form W-3? . . . . . .', '. . 5 Where Should You File Form 944-X? . . . . . . .', '. . . . . 7 Where Should You File Form 941-X? . . . .', '. . . . . . . . 2 Who Needs To File Form W-2 and Form W-3? .', '. . . 4 When Should You File Form 944-X? . . . . . .', '. . . . . . . . 4 When Should You File Form 944-X? .', '. 6 When Should You File Form 941-X? . . . . . . . .', '. . . . . . 6 When Should You File Form 941-X? . . .', '. . . . . 5 How Should You Complete Form 94

In [ ]:
# Evaluation of RAG model using RAGAS
!pip install -U ragas datasets evaluate langchain openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.1/606.1 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
from transformers import pipeline

# Define a prompt wrapper
def generate_qa_prompt(chunk):
    return f"""
Given the following IRS document excerpt, generate a question someone might ask and provide a concise answer.

Excerpt:
\"\"\"{chunk}\"\"\"

Format:
Question: <your question>
Answer: <your answer>
"""

# Loop over chunks
qa_pairs = []

for chunk in df.sample(10, random_state=42)["sentence_chunk"].tolist():
    prompt = generate_qa_prompt(chunk)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = llm_model.generate(**inputs, temperature=0.7, max_new_tokens=256)
    decoded = tokenizer.decode(output[0])

    # Try to parse output into Q/A
    try:
        question = decoded.split("Question:")[1].split("Answer:")[0].strip()
        answer = decoded.split("Answer:")[1].strip()
        qa_pairs.append({
            "question": question,
            "ground_truth": answer,
            "answer": answer,  # You can use the same for now or replace with RAG output
            "contexts": [chunk]
        })
    except Exception as e:
        print(f"Failed to parse output: {decoded}\nError: {e}")

# Show some QA pairs
import pandas as pd
qa_df = pd.DataFrame(qa_pairs)
qa_df.head()
